In [34]:
import pandas as pd
import networkx as nx
from networkx.readwrite import json_graph
import json
import random

In [35]:
nodes_path = "../file_data/fb-pages-tvshow.nodes"
edges_path = "../file_data/fb-pages-tvshow.edges"

nodes_df = pd.read_csv(
    nodes_path, sep=",", header=None, names=["node", "name", "new_id"], skiprows=1
)
edges_df = pd.read_csv(edges_path, sep=",", header=None, names=["source", "target"])

nodes_df["new_id"] = nodes_df["new_id"].astype(str)
edges_df["source"] = edges_df["source"].astype(str)
edges_df["target"] = edges_df["target"].astype(str)

In [36]:
G = nx.Graph()

for index, row in nodes_df.iterrows():
    G.add_node(row["new_id"], name=row["name"], original_id=row["node"])


for index, row in edges_df.iterrows():
    G.add_edge(row["source"], row["target"])

In [37]:
max_cliques = list(nx.find_cliques(G))
print(len(max_cliques))

6355


In [38]:
def generate_community_names(num_cliques):
    prefixes = [
        "Neo",
        "Alt",
        "Hyper",
        "Meta",
        "Quantum",
        "Terra",
        "Cyber",
        "Omega",
        "Solar",
        "Astra",
    ]
    roots = [
        "Gen",
        "Net",
        "Sphere",
        "Cluster",
        "Link",
        "Wave",
        "Flow",
        "Pulse",
        "Code",
        "Echo",
    ]
    suffixes = ["ium", "on", "os", "ix", "ia", "or", "an", "is", "us", "er"]

    community_names = set()

    while len(community_names) < num_cliques:
        name = (
            f"{random.choice(prefixes)}{random.choice(roots)}{random.choice(suffixes)}"
        )
        name += f" {random.randint(1000, 9999)}"
        if(name not in community_names):
            community_names.add(name)
    print("nomi generati")
    return list(community_names)

In [39]:
def assign_clique_names_to_graph(graph, cliques):
    num_cliques = len(cliques) 
    community_names = generate_community_names(
        num_cliques
    )
    for i, clique in enumerate(cliques):
        clique_name = community_names[i]
        saved_clique = clique
        cliques[i] = {"name": f"{clique_name}" , "nodes": clique}
        for node in saved_clique:
            if not "aggregation" in graph.nodes[node]:
                graph.nodes[node]["aggregation"] = []
            graph.nodes[node]["aggregation"].append(f"{clique_name}")
            if "community_name" not in graph.nodes[node]:
                graph.nodes[node]["community_name"] = clique_name
    for edge in graph.edges:
        source = edge[0]
        graph.edges[edge]["community_name"] = graph.nodes[source]["community_name"]
    print("nomi assegnati")

In [40]:
assign_clique_names_to_graph(G, max_cliques)
for node in G.nodes(data=True):
    print(f"Node {node[0]}: {node[1]['community_name']}")

nomi generati
nomi assegnati
Node 1852: AltLinkium 4582
Node 592: HyperEchois 8805
Node 1370: TerraSphereer 4235
Node 1263: NeoNetos 1062
Node 3042: QuantumGenon 3659
Node 3131: SolarFlower 7533
Node 2144: MetaFlowan 4949
Node 3350: QuantumWaveon 3152
Node 2280: OmegaFlowan 2048
Node 1190: MetaLinkus 1889
Node 49: AstraGenon 7097
Node 1581: QuantumSphereix 3229
Node 2134: AltEchois 6260
Node 236: SolarCodeix 1050
Node 2086: TerraClusteror 6968
Node 156: SolarNetis 1532
Node 392: HyperFlowus 9694
Node 3726: HyperGenia 3298
Node 563: SolarLinker 8078
Node 884: CyberGenor 2824
Node 59: CyberGenix 7824
Node 3056: QuantumFlowis 9254
Node 3787: AltLinkos 2522
Node 3295: AstraNeton 7309
Node 961: QuantumGenia 1322
Node 3210: HyperCodeia 3293
Node 666: HyperLinkon 8324
Node 3665: NeoCodeix 4607
Node 2956: NeoCodeor 1610
Node 3067: MetaFlowos 4695
Node 1377: NeoLinkor 4838
Node 1636: QuantumFlower 4733
Node 929: SolarClusteron 5958
Node 1427: CyberLinkon 6018
Node 532: SolarPulseium 2326
Node 1

In [41]:
def create_clique_graph(graph, cliques):
    cliqueGraph = nx.Graph()
    link_map = {}  
    clique_map = {}  
    for idx, clique in enumerate(cliques):
        clique_name = clique["name"]
        cliqueGraph.add_node(
            clique_name,
            children=[node for node in clique["nodes"] if node not in clique_map],
        )
        for node in clique["nodes"]:
            clique_map[node] = clique_name
    for edge in graph.edges():
        source_node, target_node = edge

        source_clique = clique_map.get(source_node)
        target_clique = clique_map.get(target_node)
        print(source_clique, target_clique)
        if source_clique and target_clique and source_clique != target_clique:
            link_key = f"{source_clique}-{target_clique}"
            reverse_link_key = f"{target_clique}-{source_clique}"

            if link_key not in link_map and reverse_link_key not in link_map:
                cliqueGraph.add_edge(source_clique, target_clique)
                link_map[link_key] = True

    return cliqueGraph

In [42]:
cliques_graph = create_clique_graph(G, max_cliques)

AltLinkium 4582 OmegaClusterus 1745
SolarNetis 1532 TerraCodeor 4416
SolarNetis 1532 SolarNetis 1532
SolarNetis 1532 TerraCodeor 4416
SolarNetis 1532 TerraCodeor 4416
SolarNetis 1532 SolarNetis 1532
HyperPulseus 8990 SolarGenix 5817
HyperPulseus 8990 OmegaClusteran 9814
HyperPulseus 8990 AltClusteria 6978
HyperPulseus 8990 NeoPulseus 8031
HyperPulseus 8990 SolarNeter 1760
HyperPulseus 8990 TerraSphereer 4235
HyperPulseus 8990 OmegaClusteran 9814
HyperPulseus 8990 HyperGenix 9281
HyperPulseus 8990 OmegaClusteran 9814
HyperPulseus 8990 QuantumSphereis 9582
HyperPulseus 8990 OmegaClusteran 9814
HyperPulseus 8990 OmegaClusteran 9814
HyperPulseus 8990 HyperNetix 3921
HyperPulseus 8990 MetaGenis 3298
HyperPulseus 8990 CyberEchoon 7891
HyperPulseus 8990 NeoPulseus 8031
HyperPulseus 8990 TerraGener 3137
HyperPulseus 8990 HyperNetix 3921
HyperPulseus 8990 HyperNetix 3921
HyperPulseus 8990 HyperClusteran 2766
HyperPulseus 8990 TerraPulseium 7405
HyperPulseus 8990 CyberCodeix 3533
HyperPulseus 89

In [43]:
graph_data = json_graph.node_link_data(G)
cliques_graph_data = json_graph.node_link_data(cliques_graph)
output_data= {
    "graph_data": graph_data,
    "max_cliques": max_cliques,
    "cliques_graph_data": cliques_graph_data
}

In [44]:
with open("../graph_data/graph_data_tvshow_cliques.json", "w") as f:
    json.dump(output_data, f, indent=4)